In [3]:
import logging

import hydra

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

# from .erc import KEMDy19Dataset

In [4]:
with hydra.initialize(version_base=None, config_path="./config"):
    cfg = hydra.compose(config_name="config", overrides={"dataset.generate_csv=True"})
dataset = hydra.utils.instantiate(cfg.dataset)

InstantiationException: Error locating target 'erc.datasets.KEMDy19Dataset', set env var HYDRA_FULL_ERROR=1 to see chained exception.
full_key: dataset

In [ ]:
import matplotlib.pyplot as plt 
from matplotlib import cm


COLOR_MAP  = cm.get_cmap('inferno', 7)  # visualize with the colormaps / option 'inferno' 'set3'ArithmeticError
